<a href="https://colab.research.google.com/github/riyash0109/MachineLearning/blob/main/gradio1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio

     |████████████████████████████████| 5.3 MB 32.2 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
     |████████████████████████████████| 2.3 MB 73.8 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 212 kB 87.0 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 270 kB 93.3 MB/s 
     |████████████████████████████████| 84 kB 5.0 MB/s 
     |████████████████████████████████| 112 kB 81.1 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 11.2 MB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 594 kB 77.2 MB/s 
     |████████████████████████████████| 856 kB 68.9 MB/s 
     |████████████████████████████████| 4.0 MB 57.8 MB/s 


In [ ]:
import gradio as gr
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import sklearn

In [ ]:
def encode_ages(df2): # Encoding ages 
    df2.age = df2.age.fillna(-0.5)
    bins = (-1,50,60,80,120)
    categories = pd.cut(df2.age, bins, labels=False)
    df2.age = categories
    return df2

def encode_ws(df2): # Encoding wallmotion-scores
    df2.ws = df2.ws.fillna(-0.5)
    bins = (-1, 6, 8, 12, 15, 31, 1000)
    categories = pd.cut(df2.ws, bins, labels=False)
    df2.ws = categories
    return df2

def encode_sex(df2):
    mapping = {"male": 1, "female": 2}
    return df2.replace({'group': mapping})


In [ ]:
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/content/echocardiogram.csv')

df2 = df.dropna()
df2.rename(columns={'wallmotion-score': 'ws'}, inplace=True)

def transform_features(df2):
    df2 = encode_ages(df2)
    df2 = encode_ws(df2)
    df2 = encode_sex(df2)
    return df2

train = df2[['age','ws','group','survival','alive']]
train = transform_features(train)

X_all = train.drop(['alive','survival'], axis=1)
y_all = train['alive']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))

accuracy:  0.6923076923076923


In [ ]:
df2.head()

,survival,alive,age,pericardialeffusion,fractionalshortening,epss,lvdd,ws,wallmotion-index,mult,name,group,aliveat1
0,11.0,0.0,71.0,0.0,0.260,9.000,4.600,14.0,1.00,1.000,name,1,0.0
1,19.0,0.0,72.0,0.0,0.380,6.000,4.100,14.0,1.70,0.588,name,1,0.0
2,16.0,0.0,55.0,0.0,0.260,4.000,3.420,14.0,1.00,1.000,name,1,0.0
3,57.0,0.0,60.0,0.0,0.253,12.062,4.603,16.0,1.45,0.788,name,1,0.0
4,19.0,1.0,57.0,0.0,0.160,22.000,5.750,18.0,2.25,0.571,name,1,0.0


In [ ]:
def predict_survival(group, age, ws):
    df2 = pd.DataFrame.from_dict({'group': [group], 'age': [age], 'ws': [ws]})
    df2 = encode_sex(df2)
    df2 = encode_ws(df2)
    df2 = encode_ages(df2)
    pred = clf.predict_proba(df2)[0]
    return {'Alive': pred[0], 'Death': pred[1]}


In [ ]:
group = gr.inputs.Radio(['male', 'female'], label="Sex")
age = gr.inputs.Slider(minimum=0, maximum=120, default=22, label="Age")
ws = gr.inputs.Slider(minimum=0, maximum=1000, default=100, label="Wallmotion-score")

gr.Interface(predict_survival, [group, age, ws], "label", live=True).launch();

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://26988.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
